In [2]:
from grobid_client.grobid_client import GrobidClient

client = GrobidClient(config_path="./config.json", check_server=False)
client.process("processFulltextDocument", "./test_pdf", output="./test_out/", 
                      consolidate_citations=True, tei_coordinates=True, force=True, n=20)

KeyboardInterrupt: 